In [2]:
import gensim
from gensim.models.keyedvectors import KeyedVectors as wk

model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
#Short cleanup: grep -v -e "juice" -e "carolina" -e "jam" -e "vapor" -e "drink" -e "space" -e "phone" -e "drink" relata_concatvectors.csv > cleaned_relata_concat.csv


In [80]:
import pandas as pd
df_relations = pd.read_csv('cleaned_relata_concat.csv', sep='\t', usecols=[0,1], names=["relations", "vectors"]) # we wont use cosine sim score here


In [81]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
from keras.utils import np_utils
from keras.layers import Dense, Dropout
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression

print('import done')


import done


In [115]:
Y_list = list(df_relations["relations"])
#set_Y  = list(set(Y_list))

le = LabelEncoder()
Y_test = le.fit_transform(Y_list)
num_classes = len (set(Y_test) ) 

print ( Y_test )
#Y_arr = array(Y_test)
Y_arr = np_utils.to_categorical(Y_test, num_classes)
print (Y_arr.shape)


[5 8 8 ... 8 1 8]
(13074, 10)


In [113]:
import operator
import collections

print (Y_arr[:10])
print(np.argmax(Y_arr,axis=1))

le_rel_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print (le_rel_mapping)
sorted_x = sorted(le_rel_mapping.items(), key=operator.itemgetter(1))
dict_x = collections.OrderedDict(sorted_x)
print(dict_x.keys())


[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
[5 8 8 ... 8 1 8]
{'IsA': 2, 'synonym': 9, 'antonym': 5, 'memberof': 7, 'MadeOf': 3, 'entails': 6, 'random': 8, 'PartOf': 4, 'HasProperty': 1, 'HasA': 0}
odict_keys(['HasA', 'HasProperty', 'IsA', 'MadeOf', 'PartOf', 'antonym', 'entails', 'memberof', 'random', 'synonym'])


In [83]:
print (Y_test.shape)

(13074,)


In [60]:
import re
from numpy  import array

X_set_raw = list(df_relations["vectors"])
X_set = []

for vector_string in X_set_raw:
    nums_only = vector_string.strip("[] \n")
    array_ints = [x for x in nums_only.split(" ")]
    str_list = list(filter(None, array_ints)) 
    float_list = [float(i) for i in str_list]
    numpy_arr = array(float_list)
    #print (numpy_arr)
    X_set.append( numpy_arr )

X_arr = array(X_set)



In [70]:

print (X_arr.shape)

(13074, 600)


In [117]:
train_num = 1000
x_test = X_arr[:train_num]
y_test = Y_arr[:train_num]
y_test_nums = Y_test[:train_num]
x_train = X_arr[train_num:]
y_train = Y_arr[train_num:]

print (x_test, y_test)

[[ 0.2578125  -0.25195312  0.04418945 ...  0.13867188 -0.01611328
  -0.25      ]
 [ 0.2578125  -0.25195312  0.04418945 ... -0.05200195 -0.00585938
  -0.15625   ]
 [ 0.11376953 -0.09570312  0.08691406 ... -0.08447266  0.08935547
  -0.11083984]
 ...
 [ 0.05566406  0.12695312  0.16308594 ...  0.05737305 -0.01434326
   0.10986328]
 [ 0.05566406  0.12695312  0.16308594 ...  0.09521484  0.0546875
   0.12060547]
 [ 0.05566406  0.12695312  0.16308594 ...  0.19433594  0.16796875
  -0.04956055]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [100]:
def get_simple_model():
    model = Sequential()
    model.add(Dense(520, activation='relu', input_dim=600 ))
    model.add(Dropout(0.2))
    model.add(Dense(250, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc',metrics.categorical_accuracy])
    print('compile done')
    return model

  
def check_model(model,x_train,y_train, x_val,y_val):
    model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=1,validation_data=(x_val, y_val))
    
m = get_simple_model()
check_model(m,x_train, y_train, x_test,y_test)

compile done
Train on 12074 samples, validate on 1000 samples
Epoch 1/5
12074/12074 [==============================] - 4s 340us/step - loss: 1.3170 - acc: 0.5618 - categorical_accuracy: 0.5618 - val_loss: 1.0738 - val_acc: 0.6280 - val_categorical_accuracy: 0.6280
Epoch 2/5
12074/12074 [==============================] - 3s 280us/step - loss: 0.9860 - acc: 0.6522 - categorical_accuracy: 0.6522 - val_loss: 0.9843 - val_acc: 0.6540 - val_categorical_accuracy: 0.6540
Epoch 3/5
12074/12074 [==============================] - 4s 299us/step - loss: 0.7638 - acc: 0.7300 - categorical_accuracy: 0.7300 - val_loss: 0.9564 - val_acc: 0.6920 - val_categorical_accuracy: 0.6920
Epoch 4/5
12074/12074 [==============================] - 3s 280us/step - loss: 0.5632 - acc: 0.8023 - categorical_accuracy: 0.8023 - val_loss: 1.0139 - val_acc: 0.6860 - val_categorical_accuracy: 0.6860
Epoch 5/5
12074/12074 [==============================] - 3s 269us/step - loss: 0.4059 - acc: 0.8558 - categorical_accuracy: 0.

In [119]:
import numpy as np

from sklearn.metrics import accuracy_score,classification_report

predictions = m.predict(x_test)
#print ("Predictions: ", predictions)
labels = np.argmax(predictions, axis=-1)
#print (le.inverse_transform(labels))
print (Y_test)

target_names = ['HasA', 'HasProperty', 'IsA', 'MadeOf', 'PartOf', 'antonym', 'entails', 'memberof', 'random', 'synonym']  # 0 begin, 1 end, 2 blank

print( classification_report( y_test_nums, labels, target_names = target_names ) )

print(labels[-835:])

#print(Yvalid[-835:])

[5 8 8 ... 8 1 8]
             precision    recall  f1-score   support

       HasA       0.65      0.58      0.61        53
HasProperty       0.70      0.81      0.75        85
        IsA       0.59      0.52      0.55       141
     MadeOf       0.47      0.53      0.50        17
     PartOf       0.52      0.58      0.55        43
    antonym       0.57      0.44      0.50        89
    entails       0.00      0.00      0.00         3
   memberof       0.78      0.85      0.81       501
     random       0.29      0.22      0.25        68

avg / total       0.67      0.69      0.68      1000

[4 8 9 8 9 9 1 8 8 9 8 8 8 8 8 9 2 8 8 8 1 8 8 1 0 2 8 8 8 8 0 8 8 8 4 9 8
 8 0 0 0 1 1 0 8 2 8 1 8 8 0 8 0 8 8 0 1 8 8 8 2 8 1 8 8 8 8 8 1 8 8 2 8 8
 4 2 8 5 5 2 8 9 8 8 1 8 8 9 8 1 9 8 2 4 1 2 8 8 8 2 2 2 2 8 8 8 8 2 2 8 8
 8 8 8 8 1 2 8 3 8 4 1 8 8 2 8 8 8 5 8 8 8 2 5 5 5 5 8 8 8 1 8 1 0 8 0 0 0
 8 8 8 0 0 8 8 1 0 8 8 8 5 8 8 4 8 3 8 8 8 0 4 8 2 0 2 8 1 8 8 8 8 8 8 8 8
 1 8 5 8 8 8 5 5 8 5 

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 9, does not match size of target_names, 10
  .format(len(labels), len(target_names))


In [99]:
########
clf_logreg = LogisticRegression()
clf_logreg.fit(x_train, y_train)
print ("Logistic regression: ", clf_logreg.score(x_test, y_test) )

ValueError: bad input shape (12074, 10)